In [164]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

For Beijing:
* Air Quality data: https://biendata.com/competition/airquality/bj/2018-04-01-0/2018-04-01-23/2k0d1d8
* Observed Meteorology: https://biendata.com/competition/meteorology/bj/2018-04-01-0/2018-04-01-23/2k0d1d8
* Meteorology Grid Data: https://biendata.com/competition/meteorology/bj_grid/2018-04-01-0/2018-04-01-23/2k0d1d8

For London:
* Air Quality data: https://biendata.com/competition/airquality/ld/2018-04-01-0/2018-04-01-23/2k0d1d8
* Meteorology Grid Data: https://biendata.com/competition/meteorology/ld_grid/2018-04-01-0/2018-04-01-23/2k0d1d8

In [ ]:
df_aq = pd.read_csv("../input/beijing_1701_1803.csv")
df_meo = pd.read_csv("../input/Beijing_historical_meo_grid.csv")
df_station = pd.read_csv("../input/Beijing_AirQuality_Stations.csv")

# df_aq.utc_time = pd.to_datetime(df_aq.utc_time) + pd.Timedelta('08:00:00')
# df_meo.utc_time = pd.to_datetime(df_meo.utc_time) + pd.Timedelta('08:00:00')
df_aq.utc_time = pd.to_datetime(df_aq.utc_time)
df_meo.utc_time = pd.to_datetime(df_meo.utc_time)
df_station.longitude = df_station.longitude.round(1)
df_station.latitude = df_station.latitude.round(1)

## Description

In [ ]:
df_aq.head()

In [ ]:
df_aq.describe()

In [ ]:
df_meo.head()

In [ ]:
df_meo.describe()

In [ ]:
df_station.head()

In [ ]:
df_station.describe()

# TODO

##  計算離每個aq最近的aq station

In [ ]:
for k, row in df_station.iterrows():
    # np.sqrt((111*(df_tmp.longitude-116.417))**2 + (74*(df_tmp.latitude-39.929))**2).nsmallest(4).index
    r = np.sqrt((111*(df_tmp.longitude-row[1]))**2 + (74*(df_tmp.latitude-row[2]))**2).nsmallest(4)
    print(k, row[0])
    print(r, "\n")

## 合併aq和station

In [ ]:
df_aq_station = pd.merge(df_aq, df_station, how='left',  left_on='stationId', right_on='Station_ID',)
del df_aq_station['Station_ID']
df_aq_station.to_csv("../input/beijing_1701_1803_station.csv", index=False)

# Workspace

In [161]:
targets = ['PM2.5', 'PM10', 'O3']

In [162]:
dfs = []
for stationId in df_aq.stationId.unique():
    df = df_aq[df_aq['stationId']==stationId]
    df = df.resample('1H', on='utc_time').sum()
    df = df.reset_index()
    df['stationId'] = stationId
    dfs.append(df)

In [166]:
df_ = pd.concat(dfs)
df_ = pd.merge(df_, df_station, 'left', left_on='stationId', right_on='Station_ID')
df_ = pd.merge(df_, df_meo, 'left', 
              left_on=['utc_time', 'longitude', 'latitude'], 
              right_on=['utc_time', 'longitude', 'latitude']
             )
_, bins = pd.cut(df_['wind_direction'], bins=16, retbins=True)
df_['wind_direction'] = pd.cut(df_['wind_direction'], bins=8, labels=False)

le_wind_direction = LabelEncoder()
le_wind_direction.fit(df_.station_type)
df_.station_type = le_wind_direction.transform(df_.station_type)

df_ = pd.get_dummies(df_, columns=['stationId'])

df_['year'] = df_.utc_time.dt.year
df_['month'] = df_.utc_time.dt.month
df_['day'] = df_.utc_time.dt.day
df_['hour'] = df_.utc_time.dt.hour
df_['weekday'] = df_.utc_time.dt.weekday

In [168]:
df = df_[pd.notnull(df_['PM2.5']) & 
          pd.notnull(df_['PM10']) &
          pd.notnull(df_['O3'])]
del df['SO2']
del df['CO']
del df['NO2']
del df['Station_ID']
del df['stationName']
del df['longitude']
del df['latitude']

In [177]:
y_col = 'PM2.5'
X_cols = [col for col in df.columns if col not in [y_col, 'stationId', 'utc_time']]

df_train = df[df.utc_time < "2018-03-01 00:00:00"]
df_test = df[df.utc_time >= "2018-03-01 00:00:00"]

X_train = df_train[X_cols]
y_train = df_train[y_col]
X_test = df_test[X_cols]
y_test = df_test[y_col]

# Modeling

In [183]:
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error
import xgboost as xgb
def SMAPE(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_pred-y_true) / (y_pred+y_true))) * 2


xgb_model = xgb.XGBRegressor(
    max_depth=5, 
    learning_rate=0.05, 
    n_estimators=200, 
    silent=False,
    n_jobs=3
)
xgb_model.fit(X_train, y_train, eval_metric=SMAPE)
y_pred = xgb_model.predict(X_test)
print(mean_squared_error(y_test, y_pred))

KeyboardInterrupt: 